# 跳跃关联动量因子

## 导入模块

In [1]:
import numpy as np
import pandas as pd
import feather

## 读入数据

### 日线数据

In [17]:
price_1d = feather.read_dataframe('../data/StockPriceK1d_20241231.feather')
price_1d = price_1d[(price_1d['date'] >= '2019-01-01') & (price_1d['date'] <= '2024-12-31')]

### 跳跃关联性

In [30]:
date = pd.to_datetime('2024-01-31')
date_str = date.strftime('%Y%m%d')
corr = feather.read_dataframe(f'../data/corr/corr_num_{date_str}.feather')
np.fill_diagonal(corr.values, 0)
corr = (
    corr
        .stack()
        .rename('corr')
        .rename_axis(['issue_i', 'issue_j'])
)
corr = corr.reset_index()
corr

,issue_i,issue_j,corr
0,000001,000001,0.000000
1,000001,000002,0.764706
2,000001,000004,0.411765
3,000001,000005,0.000000
4,000001,000006,0.411765
...,...,...,...
26112095,689009,688799,0.400000
26112096,689009,688800,0.571429
26112097,689009,688819,0.600000
26112098,689009,688981,0.457143


## 计算过去 20 天收益率

In [33]:
issue = '000001'
prc_issue = price_1d[price_1d['issue'] == issue]
prc_issue

,date,issue,preclose,open,high,low,close,numTrades,volume,value,adj,ret,is_limit_buy,is_limit_sell
4123,2019-01-02,000001,9.38,9.39,9.42,9.16,9.19,25140.0,53938632.0,4.986951e+08,108.031388,-0.020256,0.0,0.0
4124,2019-01-03,000001,9.19,9.18,9.33,9.15,9.28,19151.0,41553795.0,3.844577e+08,108.031388,0.009793,0.0,0.0
4125,2019-01-04,000001,9.28,9.24,9.82,9.22,9.75,59551.0,148115906.0,1.422150e+09,108.031388,0.050647,0.0,0.0
4126,2019-01-07,000001,9.75,9.84,9.85,9.63,9.74,34912.0,86568766.0,8.411664e+08,108.031388,-0.001026,0.0,0.0
4127,2019-01-08,000001,9.74,9.73,9.74,9.62,9.66,21454.0,40238811.0,3.892478e+08,108.031388,-0.008214,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,2024-12-25,000001,11.86,11.86,12.02,11.84,11.92,84645.0,147528294.0,1.759957e+09,127.784448,0.005059,0.0,0.0
121,2024-12-26,000001,11.92,11.92,11.93,11.78,11.86,62777.0,100007470.0,1.183746e+09,127.784448,-0.005034,0.0,0.0
122,2024-12-27,000001,11.86,11.87,11.90,11.66,11.83,71568.0,129001228.0,1.518383e+09,127.784448,-0.002530,0.0,0.0
123,2024-12-30,000001,11.83,11.78,11.97,11.78,11.95,86402.0,135184636.0,1.610892e+09,127.784448,0.010144,0.0,0.0
